# 05b - Vertex AI > Training > Custom Jobs - With Python Source Distribution

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05b`): An extension of `05a` that saves the script within a source distribution
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift.  

The source distribution is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob).

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05b...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)



-----
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05b_console.png">

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = '05b'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{BQ_DATASET}/models/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [6]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [8]:
FRAMEWORK = 'tf'
TASK = 'classification'
MODEL_TYPE = 'dnn'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [9]:
tb = aiplatform.Tensorboard.list(filter=f"labels.series={SERIES}")
if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = SERIES, labels = {'series' : f'{SERIES}'})

In [10]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/7179142426307592192'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [11]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

---
## Training

### Assemble Python File for Training

This is the training code from the notebook based training in `05` restructured as a Python Script that has parameter inputs and creates a Vertex AI Experiment run.

Create the main python trainer file as `/train.py`:

#### Review Pre-Built `05_train.py`

In [ ]:
from IPython.display import Markdown as md

with open(f'05_train.py', 'r') as file:
    data = file.read()
md("```python" + data + "```")

#### Copy Script to This Experiment

Create the main python trainer file as `/train.py`:

In [73]:
!mkdir -p {DIR}/source/trainer
!cp 05_train.py {DIR}/source/trainer/train.py

### Assemble Python Source Distribution

create `setup.py` file:

In [14]:
setupfile = f"""from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['tensorflow_io', 'google-cloud-aiplatform>={aiplatform.__version__}']

setup(
    name = 'trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)
"""
with open(f'{DIR}/source/setup.py', 'w') as f:
    f.write(setupfile)

add `__init__.py` file to the trainer modules folder:

In [15]:
!touch {DIR}/source/trainer/__init__.py

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [16]:
%cd {DIR}

!rm -f source.tar source.tar.gz
!tar cvf source.tar source
!gzip source.tar
!gsutil cp source.tar.gz {URI}/{TIMESTAMP}/source.tar.gz

temp = '../'*(DIR.count('/')+1)
%cd {temp}

/home/jupyter/vertex-ai-mlops/temp/05b
source/
source/setup.py
source/trainer/
source/trainer/__init__.py
source/trainer/train.py
Copying file://source.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.6 KiB/  2.6 KiB]                                                
Operation completed over 1 objects/2.6 KiB.                                      
/home/jupyter/vertex-ai-mlops


### Setup Training Job

In [17]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [18]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [f"{URI}/{TIMESTAMP}/source.tar.gz"],
            "python_module": "trainer.train",
            "args": CMDARGS
        }
    }
]

In [19]:
customJob = aiplatform.CustomJob(
    display_name = f'{EXPERIMENT}_{BQ_DATASET}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
)

### Run Training Job

In [20]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/5979235766188900352
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/5979235766188900352')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5979235766188900352?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+5979235766188900352
CustomJob projects/1026793852137/locations/us-central1/customJobs/5979235766188900352 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/5979235766188900352 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/5979235766188900352 current state:
JobState.JOB_STATE_PENDING


In [25]:
customJob.display_name

'05b_fraud_20220826114523'

In [26]:
customJob.resource_name

'projects/1026793852137/locations/us-central1/customJobs/5979235766188900352'

Create hyperlines to job and tensorboard here:

In [27]:
job_link = f"https://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}"
board_link = f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}"

In [28]:
print(f'Review the Job here:\n{job_link}')
print(f'Review the TensorBoard From the Job here:\n{board_link}')

Review the Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/5979235766188900352/cpu?cloudshell=false&project=statmike-mlops-349915
Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+5979235766188900352


---
## Serving

### Upload The Model

In [29]:
modelmatch = aiplatform.Model.list(filter = f'labels.series={SERIES} AND labels.experiemnt={EXPERIMENT}')
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        model = aiplatform.Model.upload(
            display_name = f'{EXPERIMENT}_{BQ_DATSET}',
            model_id = f'model_{EXPERIMENT}_{BQ_DATASET}',
            parent_model =  modelmatch[0].resource_name,
            serving_container_image_uri = DEPLOY_IMAGE,
            artifact_uri = f"{URI}/{TIMESTAMP}/model",
            is_default_version = True,
            version_aliases = [RUN_NAME],
            version_description = RUN_NAME,
            labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
        )
else:
    print('This is a new model, creating in model registry')
    model = aiplatform.Model.upload(
        display_name = f'{EXPERIMENT}_{BQ_DATASET}',
        model_id = f'model_{EXPERIMENT}_{BQ_DATASET}',
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}
    )  

This is a new model, creating in model registry
Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05b_fraud/operations/4122029853711007744
Model created. Resource name: projects/1026793852137/locations/us-central1/models/581364574163304448@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/581364574163304448@1')


**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`

**Retrieve a Model Resource**

[Resource](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
```Python
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

### Vertex AI Experiment Update and Review

In [30]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [31]:
expRun.log_params({
    'model.uri': model.uri,
    'model.display_name': model.display_name,
    'model.resource_name': model.resource_name,
    'model.version_id': model.version_id,
    'model.versioned_resource_name': model.versioned_resource_name,
    'customJobs.display_name': customJob.display_name,
    'customJobs.resource_name': customJob.resource_name,
    'customJobs.link': job_link,
    'customJobs.tensorboard': board_link
})

Complete the experiment run:

In [32]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Retrieve the experiment:

In [33]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [34]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.training.batch_size,param.nclasses,param.model.resource_name,param.model.save,param.model.uri,param.var_target,...,metric.val_loss,metric.test_accuracy,metric.test_auprc,metric.train_accuracy,time_series_metric.train_loss,time_series_metric.train_accuracy,time_series_metric.val_accuracy,time_series_metric.val_loss,time_series_metric.train_auprc,time_series_metric.val_auprc
0,experiment-05-05b-tf-classification-dnn,run-20220826114523,system.ExperimentRun,COMPLETE,100.0,2.0,projects/1026793852137/locations/us-central1/m...,gs://statmike-mlops-349915/fraud/models/05/05b...,gs://statmike-mlops-349915/fraud/models/05/05b...,Class,...,0.005385,0.999404,0.999582,0.999426,0.003469,0.999382,0.999256,0.005385,0.999657,0.999532


Review the Experiments TensorBoard to compare runs:

In [35]:
print(f"The Experiment TensorBoard Link:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{exp.name}")

The Experiment TensorBoard Link:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05b-tf-classification-dnn


In [36]:
expRun.get_time_series_data_frame()

,step,wall_time,train_loss,train_accuracy,val_accuracy,val_loss,train_auprc,val_auprc
0,1,2022-08-26 11:55:59.844000+00:00,0.026707,0.995203,0.998973,0.008288,0.998599,0.999239
1,2,2022-08-26 11:55:59.914000+00:00,0.005799,0.999154,0.999079,0.007146,0.999530,0.999287
2,3,2022-08-26 11:55:59.979000+00:00,0.004886,0.999246,0.999079,0.006510,0.999580,0.999296
3,4,2022-08-26 11:56:00.050000+00:00,0.004369,0.999320,0.999150,0.006175,0.999579,0.999390
4,5,2022-08-26 11:56:00.121000+00:00,0.004025,0.999351,0.999221,0.005954,0.999598,0.999484
5,6,2022-08-26 11:56:00.236000+00:00,0.004066,0.999316,0.999221,0.005711,0.999610,0.999484
6,7,2022-08-26 11:56:00.306000+00:00,0.003761,0.999351,0.999256,0.005636,0.999639,0.999485
7,8,2022-08-26 11:56:00.375000+00:00,0.003677,0.999404,0.999256,0.005496,0.999640,0.999485
8,9,2022-08-26 11:56:00.446000+00:00,0.003567,0.999404,0.999256,0.005399,0.999645,0.999485
9,10,2022-08-26 11:56:00.526000+00:00,0.003469,0.999382,0.999256,0.005385,0.999657,0.999532


### Compare This Run Using Experiments

Get a list of all experiments in this project:

In [37]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [38]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

Combine the runs from all experiments in SERIES into a single dataframe:

In [39]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05b-tf-classification-dnn
experiment-05-05a-tf-classification-dnn
experiment-05-05-tf-classification-dnn


Create ranks for models within experiment and across the entire SERIES:

In [40]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', 'param.model.display_name', 'param.model.version_id', metric]].copy().reset_index(drop = True)
    ranks['series_rank'] = ranks[metric].rank(method = 'dense', ascending = False)
    ranks['experiment_rank'] = ranks.groupby('experiment_name')[metric].rank(method = 'dense', ascending = False)
    return ranks.sort_values(by = ['experiment_name', 'run_name'])
    
ranks = ranker('metric.test_auprc')
ranks

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
4,experiment-05-05-tf-classification-dnn,run-20220825143943,05_fraud,1,0.999398,3.0,1.0
3,experiment-05-05-tf-classification-dnn,run-20220825161109,05_fraud,2,0.999397,4.0,2.0
2,experiment-05-05-tf-classification-dnn,run-20220825175329,05_fraud,3,0.999344,5.0,3.0
1,experiment-05-05a-tf-classification-dnn,run-20220826104731,05a_fraud,1,0.999627,1.0,1.0
0,experiment-05-05b-tf-classification-dnn,run-20220826114523,05b_fraud,1,0.999582,2.0,1.0


In [41]:
current_rank = ranks.loc[(ranks['param.model.display_name'] == model.display_name) & (ranks['param.model.version_id'] == model.version_id)]
current_rank

,experiment_name,run_name,param.model.display_name,param.model.version_id,metric.test_auprc,series_rank,experiment_rank
0,experiment-05-05b-tf-classification-dnn,run-20220826114523,05b_fraud,1,0.999582,2.0,1.0


In [42]:
print(f"The current model is ranked {current_rank['experiment_rank'].iloc[0]} within this experiment and {current_rank['series_rank'].iloc[0]} across this series.")

The current model is ranked 1.0 within this experiment and 2.0 across this series.


### Create/Retrieve The Endpoint For This Series

In [43]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}_{BQ_DATASET}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/4573537362990071808


In [44]:
endpoint.display_name

'05_fraud'

In [45]:
endpoint.traffic_split

{'6884847144017068032': 100}

In [46]:
deployed_models = endpoint.list_models()
deployed_models

[id: "6884847144017068032"
 model: "projects/1026793852137/locations/us-central1/models/model_05a_fraud"
 display_name: "05a_fraud"
 create_time {
   seconds: 1661512365
   nanos: 598780000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

### Should This Model Be Deployed?
Is it better than the model already deployed on the endpoint?

In [47]:
deploy = False
if deployed_models:
    for deployed_model in deployed_models:
        deployed_rank = ranks.loc[(ranks['param.model.display_name'] == deployed_model.display_name) & (ranks['param.model.version_id'] == deployed_model.model_version_id)]['series_rank'].iloc[0]
        model_rank = current_rank['series_rank'].iloc[0]
        if deployed_model.display_name == model.display_name and deployed_model.model_version_id == model.version_id:
            print(f'The current model/version is already deployed.')
            break
        elif model_rank <= deployed_rank:
            deploy = True
            print(f'The current model is ranked better ({model_rank}) than a currently deployed model ({deployed_rank}).')
            break
    if deploy == False: print(f'The current model is ranked worse ({model_rank}) than a currently deployed model ({deployed_rank})')
else: 
    deply = True
    print('No models currently deployed.')

The current model is ranked worse (2.0) than a currently deployed model (1.0)


### Deploy Model To Endpoint

In [49]:
if deploy:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = model.display_name,
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )
else: print(f'Not deploying - current model is worse ({model_rank}) than the currently deployed model ({deployed_rank})')

Not deploying - current model is worse (2.0) than the currently deployed model (1.0)


### Remove Deployed Models without Traffic

In [50]:
for deployed_model in endpoint.list_models():
    if deployed_model.id in endpoint.traffic_split:
        print(f"Model {deployed_model.display_name} with version {deployed_model.model_version_id} has traffic = {endpoint.traffic_split[deployed_model.id]}")
    else:
        endpoint.undeploy(deployed_model_id = deployed_model.id)
        print(f"Undeploying {deployed_model.display_name} with version {deployed_model.model_version_id} because it has no traffic.")

Model 05a_fraud with version 1 has traffic = 100


In [51]:
endpoint.traffic_split

{'6884847144017068032': 100}

In [52]:
endpoint.list_models()

[id: "6884847144017068032"
 model: "projects/1026793852137/locations/us-central1/models/model_05a_fraud"
 display_name: "05a_fraud"
 create_time {
   seconds: 1661512365
   nanos: 598780000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "1"]

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [53]:
pred = bigquery.query(query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits='TEST' LIMIT 10").to_dataframe()

In [54]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


In [55]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [56]:
instances = [json_format.ParseDict(newob, Value())]

### Get Predictions: Python Client

In [57]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[[0.999773204, 0.000226811419]], deployed_model_id='6884847144017068032', model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05a_fraud', explanations=None)

In [58]:
prediction.predictions[0]

[0.999773204, 0.000226811419]

In [59]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [60]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [61]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999773204,
      0.000226811419
    ]
  ],
  "deployedModelId": "6884847144017068032",
  "model": "projects/1026793852137/locations/us-central1/models/model_05a_fraud",
  "modelDisplayName": "05a_fraud",
  "modelVersionId": "1"
}


### Get Predictions: gcloud (CLI)

In [62]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999773204, 0.000226811419]]


---
## Remove Resources
see notebook "99 - Cleanup"